In [1]:
import pandas as pd
import gensim.models
from nltk.tokenize import word_tokenize
import nltk
nltk.download('punkt')
nltk.download('stopwords')
from nltk.corpus import stopwords
stop=stopwords.words('english')

[nltk_data] Downloading package punkt to /Users/mstudio/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/mstudio/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
folder='edgefield'

In [9]:
import spacy
nlp=spacy.load('en_core_web_sm')

In [10]:
import os
os.chdir('snippet-dedupe')

In [11]:
# df_negro=pd.read_csv('two_extended_anti_slavery_bugle_negro.csv')
df_slave=pd.read_csv('dedupe_two_extended_'+folder+'_slave'+'.csv')
df_servant=pd.read_csv('dedupe_two_extended_'+folder+'_servant'+'.csv')

In [12]:
frame=[df_slave, df_servant]

pre-processing

In [13]:
df=pd.concat(frame)

In [14]:
def lemmatization(dataframe:pd.DataFrame()):
    dataframe['stopword']=dataframe['context'].apply(lambda x:' '.join([word for word in x.split() if word not in (stop)]))
    dataframe['punct']=dataframe['stopword'].str.replace('[^\w\s]', '')
    dataframe['lower']=dataframe['punct'].str.lower()
    dataframe['lemma']=dataframe['lower'].apply(lambda row:' '.join([w.lemma_ for w in nlp(row)]))
    dataframe['token']=dataframe['lemma'].apply(word_tokenize)
    return dataframe

In [15]:
%%time
df=lemmatization(df)

/var/folders/3_/r8z51q092v566bd_r0g_zj640000gn/T/ipykernel_5508/2545040651.py:3: FutureWarning: The default value of regex will change from True to False in a future version.
  dataframe['punct']=dataframe['stopword'].str.replace('[^\w\s]', '')


CPU times: user 57 s, sys: 236 ms, total: 57.3 s
Wall time: 57.3 s


word2vec

In [16]:
from gensim.models.word2vec import Word2Vec

In [17]:
model=Word2Vec([row for row in df['token']], min_count=10, workers=3, window=5, sg=1)

In [18]:
from gensim.models import KeyedVectors
servant_slave=model.wv.most_similar(positive=['servant'], negative=['slave'], topn=100)
slave_servant=model.wv.most_similar(positive=['slave'], negative=['servant'], topn=100)

In [19]:
dataframe=pd.DataFrame(slave_servant, columns=('token', 'similarity'))
outname='slave_servant_'+folder+'_min10.csv'
outdir='../word2vec-dedupe'
if not os.path.exists(outdir):
    os.mkdir(outdir)
fullname=os.path.join(outdir, outname)
dataframe.to_csv(fullname)

In [20]:
dataframe=pd.DataFrame(servant_slave, columns=('token', 'similarity'))
outname='servant_slave_'+folder+'_min10.csv'
outdir='../word2vec-dedupe'
if not os.path.exists(outdir):
    os.mkdir(outdir)
fullname=os.path.join(outdir, outname)
dataframe.to_csv(fullname)